# Pangenomes graph database construction

## Authors
- Jérôme Arnoux, Genoscope/LABGeM - CEA, CNRS, Paris Saclay University
- Angela Bonifati, Liris CNRS, Lyon 1 University
- Alexandra Calteau, Genoscope/LABGeM - CEA, CNRS, Paris Saclay University
- Stefania Dumbrava, SAMOVAR/Inst. Poltech de Paris, ENSIIE
- Guillaume Gautreau, MetaGenoPolis, Université Paris-Saclay, INRAE, MGP

## Purpose
This notebook allows to reproduce the construction of a Neo4J graph database with 10 ESKAPE pangenome. The code behind was developped in the ICDE conference paper submisson.

## Notebook organisation
The structure of the notebook is as follows:

- Basic setup and dependencies.
- Pangenomic data import in the Neo4j database.
- Running the experimental analysis: query workload and scripts.
- Additional resources.

## Get our original data
To reproduce our result you can download this [folder](https://drive.google.com/drive/folders/1eZ7GQgU5tAgfryK31EPV6OP2wVRrj79B?usp=share_link). If you prefer to begin from scratch go to the supplementary code section. It will guide you to generate pangenomes and compute their similarities.

# Basic setup and dependencies
## Dependencies
Below is listed all dependencies. To install it, you can follow the step in the next section 'Environment configuration'. If you prefer you can construct your own environment by installing the dependencies listed below.

Use pip to install:
- dict2graph==2.0.0  # pip3 install git+https://git.connect.dzd-ev.de/dzdpythonmodules/dict2graph.git
- graphio==0.4.0

Use conda to install:
- ppanggolin==1.2.74  # You can only install dependencies because you will need to install the development version (1.2.105) after
- pyhmmer==0.6.3
- py2neo==2021.2.3
- rgi==6.0.1  # WARNING rgi is only compatible with python3.6 and all other dependencies with python3.9. It's necessary to install card in another environment (Refer to rgi [github](https://github.com/arpcard/rgi)).
- genome_updater==0.5.1

Neo4J:
- Add local DBMS with a Neo4j version of 4.4.11
- APOC 4.4.0.10 or more
- Optional : Neo4J Desktop 1.5.0

## Environment configuration
To begin, note that you must have an empty Neo4J DMBS (version 4.4.11) open and available with the APOC plugin install (version 4.4.0.10).

To execute the following script, you will need to install some packages. They're listed in the following conda environment file. The *in development* version of PPanGGOLiN is required to satisfy some feature and pangenomes compatibility.

To install the conda environment in jupyter kernel, please copy and paste the following code in your terminal:
```
conda update -n base -c defaults conda -y
conda env create --file conda-env.yml
conda init bash
conda activate pangraph
pip install --user ipykernel
python -m ipykernel install --user --name=pangraph
```

In [1]:
!git clone -b release1.3 https://github.com/labgem/PPanGGOLiN.git
!pip install PPanGGOLiN/.

fatal : le chemin de destination 'PPanGGOLiN' existe déjà et n'est pas un répertoire vide.
Processing ./PPanGGOLiN
  Preparing metadata (setup.py) ... done
  Created wheel for ppanggolin: filename=ppanggolin-1.2.105-cp39-cp39-linux_x86_64.whl size=3537950 sha256=8535ad73f66b6a80e1e9f2bf2f069a3f6683e20ac213696680649a88a452557e
  Stored in directory: /tmp/pip-ephem-wheel-cache-9wzla4e6/wheels/b8/98/cb/1dca3b3248b69f64c7949960a16d580eb98d8bf958044c04ed
Successfully built ppanggolin
  Attempting uninstall: ppanggolin
    Found existing installation: ppanggolin 1.2.105
    Uninstalling ppanggolin-1.2.105:
      Successfully uninstalled ppanggolin-1.2.105


## Library import
We import all the common required Python libraries to execute all the script

In [1]:
# default libraries
import logging
import os
from tqdm import tqdm
from pathlib import Path

# installed libraries
from py2neo import Graph

# local libraries
from script.python.utils import check_tsv_sanity

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [10]:
GBFF=Path("data/GBFF")  # Genome used to construct pangenomes
os.environ["GBFF"] = GBFF.as_posix()
PANGENOMES=Path("data/pangenomes")  # Directory with our pangenomes
os.environ["PANGENOMES"] = PANGENOMES.as_posix()
GF=Path("data/GF_fasta")  # Directory with for all pangenomes the DNA sequences of all gene families
os.environ["GF"] = GF.as_posix()
CARD=Path("data/CARD")
os.environ["CARD"] = CARD.as_posix()
SIMILARITY=Path("data/similarity")
os.environ["SIMILARITY"] = SIMILARITY.as_posix()

PANGENOMES_TSV=Path(f"{PANGENOMES}/organism_eskape.list")
SIMILARITIES=Path(f"{SIMILARITY}/similarity.tsv")

#Neo4J paramaeters
URI="bolt://localhost:7687"
USER="neo4j"
PWD="PANORAMA2022"

#Exec parameters
CPU=6
os.environ["CPU"]=str(CPU)
BATCH_SIZE=1000
#
pangenomes = check_tsv_sanity(PANGENOMES_TSV)  # Create a dictionnary with path file and information to pangenomes.
graph = Graph(uri=URI, user=USER, password=PWD)
graph.delete_all()  # make sure the graph is empty

# Pangenomic data import in the Neo4j database
## Translation pangenome in dictionnary to export in Graph database

The first step is to translate our pangenomes in a data structure adapted to export into our Graph database.

In [3]:
from script.python import Pangenome
from script.python.translate import write_families, write_organisms, write_spot, write_rgp, write_modules

def create_dict(pangenome: Pangenome) -> dict:
    """Create a dictionary with the pangenome content to export one pangenome into a graph database
    :param pangenome: A pangenome construct thanks to PPanGGOLiN

    :return: Compatible dictionary to export in graph database, corresponding to a pangenome
    """

    translate_dict = {"Pangenome": {"name": pangenome.name, "taxid": pangenome.taxid,
                                  "Family": [],
                                  "Partition": [],
                                  "Module": [],
                                  "RGP": write_rgp(parent=pangenome),
                                  "Spot": [], "Genome": []}}
    write_families(pangenome, translate_dict)
    write_organisms(pangenome, translate_dict)
    write_spot(pangenome, translate_dict)
    write_modules(pangenome, translate_dict)
    return translate_dict

## Loader configuration
To export pangenomes into our graph database we are using the package employed by the CovidGraph framework.

In [4]:
from multiprocessing import Lock

from py2neo import Node
# installed librairies
from dict2graph import Dict2graph


def custom_post_func(node: Node):
    if node is not None and node.__primarylabel__ == "Gene":
        del node["tmp_id"]
    return node


class PangenomeLoader:

    def __init__(self, pangenome_name: str, pangenome_data: dict, lock: Lock, batch_size: int = 1000):
        self.name = pangenome_name
        self.lock = lock
        self.data = pangenome_data
        self.batch_size = batch_size
        self._build_loader()

    def load(self, graph: Graph):
        assert self.lock is not None, "Lock not Initialized"
        try:
            with self.lock:
                logging.getLogger().debug("parse")
                self.loader.parse(self.data)
                logging.getLogger().debug("index")
                self.loader.create_indexes(graph)
                logging.getLogger().debug("merge")
                self.loader.merge(graph)
        except Exception as error:
            raise Exception(f"Load to Neo4j failed because : {error}")

    def _build_loader(self):
        d2g = Dict2graph()
        d2g.config_dict_primarykey_generated_hashed_attrs_by_label = {
            "Pangenome": 'AllAttributes',  # Random id
            "Family": ["name"],
            "Partition": 'AllAttributes',
            "Gene": 'AllAttributes',
            "Module": "InnerContent",
            "Spot": "AllContent",
            "RGP": "InnerContent",
            "Genome": "InnerContent",
            "Contig": "AllContent"
        }
        d2g.config_str_primarykey_generated_attr_name = "hash_id"
        d2g.config_list_blocklist_collection_hubs = [
            "PangenomeCollection",
            "FamilyCollection",
            "PartitionCollection",
            "GeneCollection",
            "NeighborCollection",
            "ModuleCollection",
            "SpotCollection",
            "RGPCollection",
            "GenomeCollection",
            "ContigCollection",
        ]
        d2g.config_dict_node_prop_to_rel_prop = {"Family": {"weight": ["NEIGHBOR"]}} #,
                                                 # "Shell": {"weight": ["NEIGHBOR"]},
                                                 # "Cloud": {"weight": ["NEIGHBOR"]}}  # ,  "partition": ["IN_MODULE"]}}
        d2g.config_dict_primarykey_attr_by_label = {"Family": ["name"],
                                                    "Gene": ["name"],
                                                    "Partition": ["partition"]}
        d2g.config_dict_reltype_override = {"PANGENOME_HAS_FAMILY": "IS_IN_PANGENOME",
                                            "FAMILY_HAS_GENE": "IS_IN_FAMILY",
                                            "FAMILY_HAS_PARTITION": "HAS_PARTITION",
                                            "FAMILY_HAS_FAMILY": "NEIGHBOR",
                                            "MODULE_HAS_FAMILY": "IS_IN_MODULE",
                                            "SPOT_HAS_RGP": "IS_IN_SPOT",
                                            "RGP_HAS_GENE": "IS_IN_RGP",
                                            "GENOME_HAS_CONTIG": "IS_IN_GENOME",
                                            "CONTIG_HAS_GENE": "IS_IN_CONTIG"}
        d2g.config_list_blocklist_reltypes = ["PANGENOME_HAS_MODULE",
                                              "PANGENOME_HAS_RGP",
                                              "PANGENOME_HAS_SPOT",
                                              "PANGENOME_HAS_GENOME"]
        d2g.config_bool_capitalize_labels = False
        d2g.config_func_node_post_modifier = custom_post_func
        d2g.config_graphio_batch_size = self.batch_size
        self.loader = d2g

## Export to Graph Database
### Export pangenomes

In [6]:
# default libraries
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager, Lock
from time import time
from datetime import timedelta

# local librairies
from script.python.utils import check_pangenome_info
from script.python.export import give_gene_tmp_id



db_loading_lock: Lock = None


def init_db_lock(lock: Lock):
    global db_loading_lock
    if db_loading_lock is None:
        db_loading_lock = lock


def load_pangenome(pangenome_name, pangenome_info, batch_size: int = 1000):
    """

    :param pangenome_name:
    :param pangenome_info:
    :param batch_size:
    :return:
    """
    logging.getLogger(f"Add {pangenome_name} to load list")
    pangenome = Pangenome(name=pangenome_name, taxid=pangenome_info["taxid"])
    pangenome.add_file(pangenome_info["path"])
    check_pangenome_info(pangenome, need_annotations=True, need_families=True, need_graph=True,
                         need_rgp=True, need_spots=True, need_modules=True, need_anntation_fam=True,
                         disable_bar=False)
    give_gene_tmp_id(pangenome)
    data = create_dict(pangenome)
    loader = PangenomeLoader(pangenome_name, data, db_loading_lock, batch_size=batch_size)
    loader.load(graph)


def load_pangenome_mp(pangenomes: dict, cpu: int = 1, batch_size: int = 1000):
    """

    :param pangenomes:
    :param cpu:
    :param batch_size:
    :return:
    """
    manager = Manager()
    lock = manager.Lock()
    with ProcessPoolExecutor(max_workers=cpu, initializer=init_db_lock, initargs=(lock,)) as executor:
        list(tqdm(executor.map(load_pangenome, pangenomes.keys(), pangenomes.values(),
                               [batch_size] * len(pangenomes)),
                  total=len(pangenomes), unit='pangenome'))

print("Begin pangenomes load")
begin_load_time = time()
load_pangenome_mp(pangenomes, CPU, BATCH_SIZE)
load_time = time() - begin_load_time
print(f"All pangenomes loaded in : {timedelta(seconds=load_time)}")

Begin pangenomes load


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:00<00:00, 152377.34gene family/s]


UNWIND $rels AS rel
MATCH (a:Family), (b:Partition)
WHERE a.hash_id = rel.start_hash_id AND b.hash_id = rel.end_hash_id
MERGE (a)-[r:HAS_PARTITION]->(b)
SET r = rel.properties RETURN count(r)
UNWIND $rels AS rel
MATCH (a:Family), (b:Gene)
WHERE a.hash_id = rel.start_hash_id AND b.hash_id = rel.end_hash_id
MERGE (a)-[r:IS_IN_FAMILY]->(b)
SET r = rel.properties RETURN count(r)
UNWIND $rels AS rel
MATCH (a:Family), (b:Family)
WHERE a.hash_id = rel.start_hash_id AND b.hash_id = rel.end_hash_id
MERGE (a)-[r:NEIGHBOR]->(b)
SET r = rel.properties RETURN count(r)
UNWIND $rels AS rel
MATCH (a:Pangenome), (b:Family)
WHERE a.hash_id = rel.start_hash_id AND b.hash_id = rel.end_hash_id
MERGE (a)-[r:IS_IN_PANGENOME]->(b)
SET r = rel.properties RETURN count(r)
UNWIND $rels AS rel
MATCH (a:Module), (b:Family)
WHERE a.hash_id = rel.start_hash_id AND b.hash_id = rel.end_hash_id
MERGE (a)-[r:IS_IN_MODULE]->(b)
SET r = rel.properties RETURN count(r)
UNWIND $rels AS rel
MATCH (a:RGP), (b:Gene)
WHERE a.hash

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [19:54<00:00, 597.43s/pangenome]


All pangenomes loaded in : 0:19:55.591789


## Export similarities

In [12]:
# installed librairies
from graphio import RelationshipSet
import pandas as pd

def load_similarities(tsv: Path, batch_size: int = 1000):
    df = pd.read_csv(filepath_or_buffer=tsv, sep="\t", header=None,
                     names=["Family_1", "Family_2", "identity", "covery"])

    is_similar_list = []
    is_similar_to = RelationshipSet('IS_SIMILAR', ['Family'], ['Family'], ['name'], ['name'])
    chunk_size = batch_size * 10
    for row in df.iterrows():
        if len(is_similar_to.relationships) >= chunk_size:
            is_similar_list.append(is_similar_to)
            is_similar_to = RelationshipSet('IS_SIMILAR', ['Family'], ['Family'], ['name'], ['name'])
        is_similar_to.add_relationship(start_node_properties={"name": row[1]['Family_1']},
                                       end_node_properties={"name": row[1]['Family_2']},
                                       properties={"identity": row[1]['identity'],
                                                   "coverage": row[1]['covery']})
    is_similar_list.append(is_similar_to)
    for sim in tqdm(is_similar_list, unit="similarities_batch", total=len(is_similar_list)):
        sim.merge(graph=graph, batch_size=batch_size)

print("Bengin load of similarities...")
begin_sim_time = time()
load_similarities(SIMILARITIES, BATCH_SIZE)
sim_time = time() - begin_sim_time
print(f"All similarities loaded in : {timedelta(seconds=sim_time)}")

Bengin load of similarities...


  0%|                                                                                                       | 0/1 [00:00<?, ?similarities_batch/s]

UNWIND $rels AS rel
MATCH (a:Family), (b:Family)
WHERE a.name = rel.start_name AND b.name = rel.end_name
MERGE (a)-[r:IS_SIMILAR]->(b)
SET r = rel.properties RETURN count(r)


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:12<00:00, 72.42s/similarities_batch]

All similarities loaded in : 0:01:12.611399


### Invert edges

In [13]:
from script.python.export import invert_edges_query


def invert_edges(edge_label: str):
    query = invert_edges_query(edge_label)
    try:
        graph.run(query)
    except Exception as errror:
        raise Exception(f"Invert edges failed because : {errror}")

print("Invert edges...")
begin_invert_time = time()
labels2invert = ["IS_IN_PANGENOME", "IS_IN_MODULE", "IS_IN_FAMILY", "IS_IN_CONTIG",
                 "IS_IN_GENOME", "IS_IN_SPOT", "IS_IN_RGP"]
for edge_label in tqdm(labels2invert, unit='label'):
    logging.getLogger().debug(f"Invert: {edge_label}")
    invert_edges(edge_label)
invert_time = time() - begin_invert_time
print(f"All edges inverted in : {timedelta(seconds=invert_time)}")

Invert edges...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:51<00:00, 24.57s/label]

All edges inverted in : 0:02:51.981150


# Running the experimental analysis: query workload and scripts

In [14]:
from time import time

def launch_query(query):
    print(query)
    try:
        res = graph.run(query)
    except Exception as errror:
        raise Exception(f"Query : '{query}' failed because of the following errror\n{errror}")
    else:
        return  res

def launch_WF():
    from script.python.wf import WF
    from statistics import mean, median, stdev
    import pandas as pd

    nb_rep = 10
    stat_dict = {}
    for q_id, query in WF.items():
        list_q_time = []
        for i in range(nb_rep):
            launch_query("CALL apoc.warmup.run()")
            begin_time = time()
            res = launch_query(query)
            q_time = time() - begin_time
            list_q_time.append(q_time)
            launch_query("CALL db.clearQueryCaches()")
        stat_dict[q_id] = [mean(list_q_time), stdev(list_q_time), median(list_q_time),
                           min(list_q_time), max(list_q_time)]
    return pd.DataFrame.from_dict(stat_dict, orient='index', columns=["Mean", "Stdev", "Mediane", "Min", "Max"])

launch_WF()

CALL apoc.warmup.run()
MATCH (p:Pangenome)<-[:IS_IN_PANGENOME]-(f:Family) WHERE f.annotation IS NOT NULL RETURN p.name, count(f)
CALL db.clearQueryCaches()
CALL apoc.warmup.run()
MATCH (p:Pangenome)<-[:IS_IN_PANGENOME]-(f:Family) WHERE f.annotation IS NOT NULL RETURN p.name, count(f)
CALL db.clearQueryCaches()
CALL apoc.warmup.run()
MATCH (p:Pangenome)<-[:IS_IN_PANGENOME]-(f:Family) WHERE f.annotation IS NOT NULL RETURN p.name, count(f)
CALL db.clearQueryCaches()
CALL apoc.warmup.run()
MATCH (p:Pangenome)<-[:IS_IN_PANGENOME]-(f:Family) WHERE f.annotation IS NOT NULL RETURN p.name, count(f)
CALL db.clearQueryCaches()
CALL apoc.warmup.run()
MATCH (p:Pangenome)<-[:IS_IN_PANGENOME]-(f:Family) WHERE f.annotation IS NOT NULL RETURN p.name, count(f)
CALL db.clearQueryCaches()
CALL apoc.warmup.run()
MATCH (p:Pangenome)<-[:IS_IN_PANGENOME]-(f:Family) WHERE f.annotation IS NOT NULL RETURN p.name, count(f)
CALL db.clearQueryCaches()
CALL apoc.warmup.run()
MATCH (p:Pangenome)<-[:IS_IN_PANGENOME]-(

,Mean,Stdev,Mediane,Min,Max
Q1,0.083915,0.046694,0.069744,0.048335,0.204392
Q2,0.056861,0.007986,0.055250,0.045735,0.074058
Q3,0.747413,0.113417,0.724246,0.657622,1.053809
Q4,1.490831,0.132928,1.450817,1.327790,1.781401
Q5,0.046255,0.005752,0.044578,0.042577,0.062443
Q6,0.044386,0.003446,0.043214,0.039686,0.050909
Q7,2.436580,0.067438,2.420400,2.357164,2.563146
Q8,1.271319,0.172032,1.198555,1.097136,1.658775
Q9,0.074360,0.015719,0.064597,0.061571,0.106834
Q10,0.071507,0.035591,0.058783,0.051264,0.170405


# Suplementary code
## Genomes downloading
Step do download genomes necessary to consctruct pangenomes. Note that genomes database are in constant evolution and your pangenome could be different than our.

In [8]:
!mkdir $GBFF
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Acinetobacter baumannii" -f assembly_report.txt,genomic.gbff.gz -l "complete genome" -o $GBFF/Acinetobacter.baumannii -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Enterobacter bugandensis" -f assembly_report.txt,genomic.gbff.gz -o $GBFF/Enterobacter.bugandensis -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Enterobacter cloacae" -f assembly_report.txt,genomic.gbff.gz -o $GBFF/Enterobacter.cloacae -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Enterobacter hormaechei_A" -f assembly_report.txt,genomic.gbff.gz -l "complete genome" -o $GBFF/Enterobacter.hormaechei_A -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Enterobacter kobei" -f assembly_report.txt,genomic.gbff.gz -o $GBFF/Enterobacter.kobei -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Enterobacter roggenkampii" -f assembly_report.txt,genomic.gbff.gz -o $GBFF/Enterobacter.roggenkampii -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Enterococcus_B faecium" -f assembly_report.txt,genomic.gbff.gz -l "complete genome" -o $GBFF/Enterococcus_B.faecium -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Klebsiella pneumoniae" -f assembly_report.txt,genomic.gbff.gz -l "complete genome" -A 600 -o $GBFF/Klebsiella.pneumoniae -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Pseudomonas aeruginosa" -f assembly_report.txt,genomic.gbff.gz -l "complete genome" -o $GBFF/Pseudomonas.aeruginosa -t $CPU
!genome_updater.sh -d genbank,refseq -M gtdb -T "s__Staphylococcus aureus" -f assembly_report.txt,genomic.gbff.gz -l "complete genome" -A 600 -o $GBFF/Staphylococcus.aureus -t $CPU

-------------------------------------------
┌─┐┌─┐┌┐┌┌─┐┌┬┐┌─┐    ┬ ┬┌─┐┌┬┐┌─┐┌┬┐┌─┐┬─┐
│ ┬├┤ ││││ ││││├┤     │ │├─┘ ││├─┤ │ ├┤ ├┬┘
└─┘└─┘┘└┘└─┘┴ ┴└─┘────└─┘┴  ─┴┘┴ ┴ ┴ └─┘┴└─
                                     v0.5.1 
-------------------------------------------
Mode: NEW 
Args: -T 's__Acinetobacter baumannii' -M 'gtdb' -t '6' -o 'data/GBFF/Acinetobacter.baumannii' -l 'complete genome' -f 'assembly_report.txt,genomic.gbff.gz' -d 'genbank,refseq'
Outp: /home/jarnoux/Projects/PanGraph-DB/data/GBFF/Acinetobacter.baumannii/
-------------------------------------
 - Database [genbank,refseq]
 - 1821534 assembly entries available

Filtering assembly summary [2022-11-21_10-51-39]
 - Downloading taxonomy (gtdb)
 - 1503993 assemblies removed not in GTDB
 - Could not retrieve 1 GTDB assemblies
 - 312124 assemblies removed not in taxids [s__Acinetobacter baumannii]
 - 5132 assemblies removed based on filters:
   valid URLs
   assembly level=complete genome
 - 285 assembly entries to download

 -

## Pangenome construction with PPanGGOLiN
### Generate list of genomes

In [20]:
! for sp in $(ls $GBFF); do for path in $(ls $GBFF/$sp/*/files/*.gbff.gz);do genome=$(echo $path | cut -d'/' -f6 | cut -d. -f1,2); echo $genome $(pwd)/$path | sed 's/\s/\t/'; done > $PANGENOMES/$sp.list; done

### Generate pangenomes

In [4]:
!ppanggolin all --anno $PANGENOMES/Acinetobacter.baumannii.list -o $PANGENOMES/Acinetobacter.baumannii -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Enterobacter.bugandensis.list -o $PANGENOMES/Enterobacter.bugandensis -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Enterobacter.cloacae.list -o $PANGENOMES/Enterobacter.cloacae -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Enterobacter.hormaechei_A.list -o $PANGENOMES/Enterobacter.hormaechei_A -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Enterobacter.kobei.list -o $PANGENOMES/Enterobacter.kobei -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Enterobacter.roggenkampii.list -o $PANGENOMES/Enterobacter.roggenkampii -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Enterococcus_B.faecium.list -o $PANGENOMES/Enterococcus_B.faecium -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Klebsiella.pneumoniae.list -o $PANGENOMES/Klebsiella.pneumoniae -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Pseudomonas.aeruginosa.list -o $PANGENOMES/Pseudomonas.aeruginosa -c $CPU -f
!ppanggolin all --anno $PANGENOMES/Staphylococcus.aureus.list -o $PANGENOMES/Staphylococcus.aureus -c $CPU -f

2022-11-21 12:00:06 utils.py:l116 INFO	Command: /home/jarnoux/anaconda3/envs/pangraph/bin/ppanggolin all --anno data/pangenomes/Acinetobacter.baumannii.list -o data/pangenomes/Acinetobacter.baumannii -c 6 -f
2022-11-21 12:00:06 utils.py:l117 INFO	PPanGGOLiN version: 1.2.105
2022-11-21 12:00:06 annotate.py:l448 INFO	Reading data/pangenomes/Acinetobacter.baumannii.list the list of organism files ...
100%|███████████████████████████████████████| 285/285 [00:34<00:00,  8.20file/s]
2022-11-21 12:00:43 annotate.py:l469 INFO	gene identifiers used in the provided annotation files were unique, PPanGGOLiN will use them.
2022-11-21 12:00:43 writeBinaries.py:l932 INFO	Writing genome annotations...
100%|█████████████████████████████████████| 285/285 [00:02<00:00, 96.23genome/s]
2022-11-21 12:00:47 writeBinaries.py:l950 INFO	writing the protein coding gene dna sequences
100%|███████████████████████████| 1045048/1045048 [00:01<00:00, 651535.64gene/s]
2022-11-21 12:00:53 writeBinaries.py:l993 INFO	Don

## Pangenome AMR annotation with CARD database and RGI
### Get gene faimilies reference sequences

In [6]:
!mkdir $GF
!ppanggolin fasta -p $PANGENOMES/Acinetobacter.baumannii/pangenome.h5 -o $GF/Acinetobacter.baumannii --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Enterobacter.bugandensis/pangenome.h5 -o $GF/Enterobacter.bugandensis --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Enterobacter.cloacae/pangenome.h5 -o $GF/Enterobacter.cloacae --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Enterobacter.hormaechei_A/pangenome.h5 -o $GF/Enterobacter.hormaechei_A --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Enterobacter.kobei/pangenome.h5 -o $GF/Enterobacter.kobei --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Enterobacter.roggenkampii/pangenome.h5 -o $GF/Enterobacter.roggenkampii --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Enterococcus_B.faecium/pangenome.h5 -o $GF/Enterococcus_B.faecium --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Klebsiella.pneumoniae/pangenome.h5 -o $GF/Klebsiella.pneumoniae --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Pseudomonas.aeruginosa/pangenome.h5 -o $GF/Pseudomonas.aeruginosa --gene_families all -f
!ppanggolin fasta -p $PANGENOMES/Staphylococcus.aureus/pangenome.h5 -o $GF/Staphylococcus.aureus --gene_families all -f

mkdir: impossible de créer le répertoire « data/GF_fasta »: Le fichier existe
2022-11-21 13:34:28 utils.py:l116 INFO	Command: /home/jarnoux/anaconda3/envs/pangraph/bin/ppanggolin fasta -p data/pangenomes/Acinetobacter.baumannii/pangenome.h5 -o data/GF_fasta/Acinetobacter.baumannii --gene_families all -f
2022-11-21 13:34:28 utils.py:l117 INFO	PPanGGOLiN version: 1.2.105
2022-11-21 13:34:28 readBinaries.py:l51 INFO	Getting the current pangenome status
2022-11-21 13:34:28 readBinaries.py:l505 INFO	Reading pangenome annotations...
100%|███████████████████████████████████| 284/284 [00:15<00:00, 17.84organism/s]
2022-11-21 13:34:47 readBinaries.py:l519 INFO	Reading pangenome gene families...
100%|█████████████████████████| 14427/14427 [00:00<00:00, 67431.59gene family/s]
2022-11-21 13:34:51 writeSequences.py:l95 INFO	Writing all of the representative nucleotide sequences of the gene families...
2022-11-21 13:34:51 readBinaries.py:l130 INFO	Extracting and writing CDS sequences from a data/pan

### Annotate with RGI
RGI is only compatible with python version 3.6. It's why it's not possible to run it in the current notebook.
To annotate your pangenome you have to follow the installation instruction [here](https://github.com/arpcard/rgi#id52).

To annotate a pangenome you can use the following commands :
```
rgi load --card_json /path/to/card.json --local
rgi main -i $GF/speciesall_nucleotide_families.fasta -o $CARD/species/card_res -n $CPU --include_nudge
```
When it's done for all pangenome, you can run the next block of code.

In [3]:
import tables
from script.python import Pangenome
from script.python.utils import check_pangenome_info
from script.python.annot import card_parse, annotation_to_families, write_gene_fam_annot

pangenome_dir = [pan for pan in PANGENOMES.iterdir() if pan.is_dir()]
for card, pan in zip(CARD.iterdir(), pangenome_dir):
    current_pangenome = Pangenome(name=pan.name)
    current_pangenome.add_file(f'{pan}/pangenome.h5')
    check_pangenome_info(current_pangenome, need_annotations=True, need_families=True, need_graph=True,
                         need_rgp=True, need_spots=True, need_modules=True, need_anntation_fam=True,
                         disable_bar=False)
    card_data = card_parse(f'{card}/card_res.txt')
    annotation_to_families(card_data, pangenome=current_pangenome, source='CARD')
    h5f = tables.open_file(current_pangenome.file, "a")
    write_gene_fam_annot(current_pangenome, h5f, force=True, disable_bar=False)
    h5f.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4260/4260 [00:00<00:00, 375301.62module/s]
0gene family [00:00, ?gene family/s]


Erasing the formerly computed gene family annotations...


99annot [00:00, 17419.81annot/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 8197/8197 [00:00<00:00, 430889.96module/s]
There is no annotation in Klebsiella.pneumoniae
140annot [00:00, 19160.20annot/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2635/2635 [00:00<00:00, 349083.73module/s]
There is no annotation in Enterobacter.bugandensis
49annot [00:00, 21665.71annot/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 3789/3789 [00:00<00:00, 213032.41module/s]
There is no annotation in Acinetobacter.baumannii
97annot [00:00, 31782.48annot/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5017/5017 [00:00<00:00, 358798.65module/s]
There is no annotation in Enterobacter.cloacae
117annot [00:00, 32443.05annot/s]
100%|████████████████████████████████████████████

# Compute similarity
## Align sequence

In [14]:
!cat $GF/*/all_nucleotide_families.fasta > $SIMILARITY/all_nucleotide_families.fasta
!mmseqs createdb $SIMILARITY/all_nucleotide_families.fasta $SIMILARITY/sequenceDB
!mmseqs search $SIMILARITY/sequenceDB $SIMILARITY/sequenceDB $SIMILARITY/alignDB /tmp/ --search-type 3 --threads $CPU
!mmseqs convertalis $SIMILARITY/sequenceDB $SIMILARITY/sequenceDB $SIMILARITY/alignDB $SIMILARITY/align.tsv --format-output query,target,pident,qcov,tcov --threads $CPU

convertalis data/similarity/sequenceDB data/similarity/sequenceDB data/similarity/alignDB data/similarity/align.tsv --format-output query,target,pident,qcov,tcov --threads 6 

MMseqs Version:        	14.7e284
Substitution matrix    	aa:blosum62.out,nucl:nucleotide.out
Alignment format       	0
Format alignment output	query,target,pident,qcov,tcov
Translation table      	1
Gap open cost          	aa:11,nucl:5
Gap extension cost     	aa:1,nucl:2
Database output        	false
Preload mode           	0
Search type            	0
Threads                	6
Compressed             	0
Verbosity              	3

[=================================================================] 100.00% 25.97K 0s 44ms     
Time for merging to align.tsv: 0h 0m 0s 11ms
Time for processing: 0h 0m 0s 68ms


## Filter and parse alignment results

In [11]:
def parse(file: Path, identity: float, coverage: float):
    df = pd.read_csv(filepath_or_buffer=file, sep="\t", header=None,
                     names=["GF_1", "GF_2", "identity", "qcov", "tcov"])
    df = df[df['GF_1'] != df['GF_2']]
    df = df[df['identity'] > identity]
    df = df[(df['qcov'] > coverage) & (df['tcov'] > coverage)]
    filter_df = pd.concat([df[['GF_1', 'GF_2', 'identity']],
                           df[['qcov', 'tcov']].min(axis=1).apply(lambda x: x*100)], axis=1)
    filter_df.columns = ['GF_1', 'GF_2', 'identity', 'coverage']
    return filter_df

align_file = Path(f'{SIMILARITY}/align.tsv')
filter_df = parse(align_file, identity=30, coverage=0.8)
filter_df.to_csv(f'{SIMILARITY}/similarity.tsv', sep="\t", header=None, index=False)

# Clean to relaunch

In [ ]:
!rm -rf PPanGGOLiN
!conda env remove -n pangraph

# References
1. S. Sakr _qnd al_. “The future is big graphs: a community view on graph processing systems,” Commun. ACM, vol. 64, no.9, pp. 62–71, 2021.
2. G. Gautreau _and al_. “PPanGGOLiN: Depicting microbial diversity via a partitioned pangenome graph,” vol. 16, no. 3, p. e1007732, publisher: Public Library of Science. [https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007732](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007732)